In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import os

from main import DCTrader, Backtest, DCTracker
from trainer import GATrainer, PSOTrainer

In [3]:
from strategies import St1, St2, St3, St4, St5, St6, St7, St8

ticker = 'AAPL'

thresholds = [0.00098, 0.0022, 0.0048, 0.0072, 0.0098, 0.0122, 0.0155, 0.017, 0.02, 0.0255]
strategies = [St1, St2, St3, St4, St5, St6, St7, St8]

# Modo entrenamiento (precomputa estados)
trader = DCTrader(
    ticker=ticker,
    thresholds=thresholds,
    strategies=strategies,
    is_train=True,
    start_date='2016-01-01',
    end_date='2024-01-01'
)

print(f"Trader creado para {ticker}")
print(f"Thresholds: {thresholds}")
print(f"Datos cargados: {len(trader.prices)} días")

AAPL: 2012 loaded days | 2016-01-04 to 2023-12-29
Trader creado para AAPL
Thresholds: [0.00098, 0.0022, 0.0048, 0.0072, 0.0098, 0.0122, 0.0155, 0.017, 0.02, 0.0255]
Datos cargados: 2012 días


In [4]:
total_weights = len(trader.combination_matrix)
total_weights

70

In [5]:
# import numpy as np
# n_strats = len(strategies)
# n_ths = len(thresholds)
# weights = np.ones((n_strats, n_ths)) / 70  # Igual peso en todos θ por estrategia
# results = trader.backtest(weights=weights)
# print(results)  # Debe dar SR >0 y trades

In [ ]:
trader.fit()

Iniciando entrenamiento GA | Población: 150 | Generaciones: 50
Número total de pesos a optimizar: 70
Gen   1 | SR: -0.016 | RoR: -0.212 | ToR: 0.16402 | Trades: 330 | Fit: -0.163
Gen  10 | SR: -0.002 | RoR: -0.111 | ToR: 0.15755 | Trades: 317 | Fit: -0.124
